In [42]:
f = open(r'cifar-10-batches-py/1/batches.meta', mode='r')

In [43]:
import matplotlib.pyplot as plt
import numpy as np

import keras
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Activation,Flatten,BatchNormalization
import warnings
warnings.filterwarnings('ignore')

In [44]:
import pickle

def unpickle(file):
      with open(file, 'rb') as fo:
              dict = pickle.load(fo, encoding='bytes')
      return dict

In [45]:
# 测试集文件
test_batch = unpickle(r'cifar-10-batches-py\test_batch')

X_test = np.array(test_batch[b'data'])
y_test = np.array(test_batch[b'labels']) # list type

In [46]:
# 训练集文件
train_files = ['data_batch_'+str(i) for i in range(1,6)]

train_data_list,train_label_list = [],[]

for f in train_files:
    
    fpath = r'cifar-10-batches-py/' + f
    batch_dict = unpickle(fpath)
    
    batch_data = batch_dict[b'data']
    batch_labels = batch_dict[b'labels']
    train_data_list.append(batch_data)
    train_label_list.append(batch_labels)

X_train = np.concatenate(train_data_list, axis = 0)
y_train = np.concatenate(train_label_list, axis = 0)

In [50]:
label_names_batch = unpickle(r'cifar-10-batches-py/1/batches.meta')
label_names = label_names_batch[b'label_names']
label_names = [l.decode("utf-8") for l in label_names]

In [51]:
print('训练集特征：',X_train.shape,'，训练集label',y_train.shape)
print('测试集特征：',X_test.shape,'，测试集label', y_test.shape)
print("类别名字：",label_names)

训练集特征： (50000, 3072) ，训练集label (50000, 10)
测试集特征： (10000, 3072) ，测试集label (10000, 10)
类别名字： ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [52]:
X_train = X_train.reshape(-1,3,32,32)
X_test = X_test.reshape(-1,3,32,32)

In [53]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 归一化
X_train /= 255
X_test /= 255

# 将class vectors转变成binary class metrics
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [54]:
# Method 1: Switch backend
keras.backend.set_image_dim_ordering('th')
print('Backend:',keras.backend.image_dim_ordering())


Backend: th


In [55]:
def base_model(opt):
    model = Sequential()
    
    # 32个卷积核(feature maps),步长为1，特征图的大小不会改变（周边补充空白），
    model.add(Conv2D(32,(3,3), padding="same", input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))

    # channel是在前面 (theano后台)
    MaxPooling2D(pool_size=(2, 2), data_format="channels_first")
    model.add(Dropout(0.25))
    
    # 64个卷积核
    model.add(Conv2D(64,(3,3),padding="same"))
    model.add(Activation('relu'))
    
    MaxPooling2D(pool_size=(2, 2), data_format="channels_first")
    model.add(Dropout(0.25))
    
    model.add(Flatten())   # Flatten layer
    model.add(Dropout(0.25))
    model.add(Dense(512))  # fully connected layer with 512 units
    model.add(Activation('relu'))
    
    model.add(Dropout(0.25))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy']) # 要优化的是准确率
    return model

In [62]:
opt1 = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
cnn2 = base_model(opt1)
cnn2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_9 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 32, 32)        18496     
_________________________________________________________________
activation_10 (Activation)   (None, 64, 32, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 64, 32, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 65536)             0         
__________

In [63]:
epochs = 12
batch_size = 100
verbose = True

In [64]:
history = cnn2.fit(X_train, y_train, 
                    epochs = 10, batch_size = 32, 
                    validation_data=(X_test,y_test), 
                    shuffle=True)

ValueError: Error when checking target: expected activation_12 to have 2 dimensions, but got array with shape (50000, 10, 2)

In [59]:
# 对样本进行测试，默认不使用正则化
score2 = cnn2.evaluate(X_test,y_test)
print("损失值为{0:.2f},准确率为{1:.2%}".format(score2[0],score2[1]))

10000/10000 [==============================] - 18s 2ms/step
损失值为1.19,准确率为64.54%
